In [1]:
import sagemaker

print(sagemaker.__version__)

session = sagemaker.Session()

2.0.0rc1


In [ ]:
%%sh
# https://s3.amazonaws.com/amazon-reviews-pds/readme.html
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz /tmp

In [2]:
input_data='s3://sagemaker-us-east-1-886035371869/amazon-reviews-camera/amazon_reviews_us_Camera_v1_00.tsv.gz'

In [ ]:
prefix = 'amazon-reviews-camera'

input_data = session.upload_data(path='/tmp/amazon_reviews_us_Camera_v1_00.tsv.gz', key_prefix=prefix)
print(input_data)

In [3]:
from sagemaker.sklearn.processing import SKLearnProcessor

role = sagemaker.get_execution_role()
sklearn_processor = SKLearnProcessor(framework_version='0.20.0',
                                     role=role,
                                     instance_type='ml.c5.2xlarge',
                                     instance_count=1)

In [4]:
%%time

from sagemaker.processing import ProcessingInput, ProcessingOutput

sklearn_processor.run(
    code='preprocessing-word2vec.py',
    
    inputs=[
        ProcessingInput(
                source=input_data,
                destination='/opt/ml/processing/input')
    ],
    
    outputs=[
        ProcessingOutput(
                output_name='train_data',
                source='/opt/ml/processing/train')
    ],
    
    arguments=[
               '--num-reviews', '100000',
               '--filename', 'amazon_reviews_us_Camera_v1_00.tsv.gz'
    ]
)


Job Name:  sagemaker-scikit-learn-2021-03-27-08-00-51-443
Inputs:  [{'InputName': 'input-1', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-886035371869/amazon-reviews-camera/amazon_reviews_us_Camera_v1_00.tsv.gz', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-886035371869/sagemaker-scikit-learn-2021-03-27-08-00-51-443/input/code/preprocessing-word2vec.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'train_data', 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-886035371869/sagemaker-scikit-learn-2021-03-27-08-00-51-443/output/train_data', 'LocalPath': '/opt/ml/processing/train', 'S3UploadMode': 'EndOfJob'}}]
.........................../miniconda3/l

In [5]:
preprocessing_job_description = sklearn_processor.jobs[-1].describe()

output_config = preprocessing_job_description['ProcessingOutputConfig']
for output in output_config['Outputs']:
    print(output['S3Output']['S3Uri'])

s3://sagemaker-us-east-1-886035371869/sagemaker-scikit-learn-2021-03-27-08-00-51-443/output/train_data
